# Pytorch 101

In [ ]:
import torch
import numpy as np

In [ ]:
tensor_2d = torch.randn(3,4)
tensor_2d

In [ ]:
tensor_3d = torch.zeros(2,3,4)
tensor_3d

In [ ]:
np1 = np.random.rand(4,5)
my_tensor = torch.tensor(np1)
my_tensor
# tensors are by default float32, but can 

## Tensor Ops

In [ ]:
import torch

In [ ]:
my_torch = torch.arange(10)
my_torch

In [ ]:
# Reshape and View
my_torch = my_torch.reshape(2,5)
my_torch
# Reshape if we dont know the number of items and udo that use -1
my_torch = my_torch.reshape(-1, 5)
my_torch

In [ ]:
my_torch2 = torch.arange(10)
my_torch3 = my_torch2.reshape(2,5)

In [ ]:
my_torch3

In [ ]:
my_torch2[1] = 4141
my_torch2

In [ ]:
my_torch3

In [ ]:
my_torch4 = torch.arange(10)
my_torch4

In [ ]:
my_torch4[4]

In [ ]:
# Grab slice
my_torch5 = torch.arange(10)
my_torch6 = my_torch5.reshape(5,2)
my_torch6[:,1]

In [ ]:
my_torch6[:,1:]

## Tensor Math Operations

In [ ]:
import torch
import numpy as np

In [ ]:
tensor_a = torch.tensor([1,2,3,4])
tensor_b = torch.tensor([5,6,7,8])

In [ ]:
tensor_a + tensor_b

In [ ]:
torch.add(tensor_a, tensor_b)

In [ ]:
tensor_b - tensor_a

In [ ]:
torch.sub(tensor_b, tensor_a)

In [ ]:
tensor_a * tensor_b

In [ ]:
torch.mul(tensor_a, tensor_b)

In [ ]:
tensor_b/tensor_a

In [ ]:
torch.div(tensor_b, tensor_a)

In [ ]:
tensor_b % tensor_a

In [ ]:
torch.remainder(tensor_b, tensor_a)

In [ ]:
# Exponenstial power
print([1**5, 2**6, 3**7, 4**8])
torch.pow(tensor_a, tensor_b)

In [ ]:
tensor_a.add(tensor_b)

In [ ]:
tensor_a.add_(tensor_b)
tensor_a

# Build Simple Neural Network with PyTorch

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Create a Model Class that inherits from nn.Module
class Model(nn.Module):
    # Input Layer (4 features of the flower) --> 
    # Hiddenlayer 1 H1 (number of neurons) --> 
    # H2 (n number of neurons) --> 
    # Output Layer (3 classes of the flower)

    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1) # Input to H1
        self.fc2 = nn.Linear(h1, h2)          # H1 to H2
        self.out = nn.Linear(h2, out_features) # H2 to Output Layer
    
    def forward(self, x):
        x = F.relu(self.fc1(x)) # Do something, if the output of something is <-, use 0, else use the output
        x = F.relu(self.fc2(x))
        x = self.out(x) # Final Layer, no activation function
        return x

In [ ]:
# create a manual seed for randomness
torch.manual_seed(41)
# Create an instance of the model
model = Model()

In [ ]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)

In [ ]:
my_df.tail()

In [ ]:
# Change last column from strings to integers
my_df['species'] = my_df['species'].replace({'setosa':0, 'versicolor':1, 'virginica':2})
my_df

In [ ]:
# Train and Test Split and make X and y as numpy arrays
X = my_df.drop('species', axis=1).values
y = my_df['species'].values
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train) #LongTensor for classification, longtensors are 64 bit integers
y_test = torch.LongTensor(y_test)

In [ ]:
# Set the creiterion to measure the error
criterion = nn.CrossEntropyLoss()
# Choose an optimizer, learning rate, lr = 0.01 if error doesnt go down after epochs, lower the lr. The lower lr, the longer it takes to run
optimizer = torch.optim.Adam((model.parameters()), lr = 0.01)

In [ ]:
# Define the number of epochs
# Epoch is one run through all the the training data
epochs = 100
losses = [] # append losses as we go through the epochs
for i in range(epochs):
    # Go forward pass and get a prediction
    y_pred = model.forward(X_train) # Get predicted results

    # Measure the loss
    loss = criterion(y_pred, y_train) # Compare predicted results to actual results

    # Keep track of losses
    losses.append(loss.detach().numpy())

    # print every 10 epochs
    if i%10==0:
        print(f'Epoch: {i} Loss: {loss}')
    
    # Do some backporpagation: Take the error rate from forward propagation and feed it back through the network to fine tune the weights.
    optimizer.zero_grad() # clears the old gradient values, ensuring gradients from the previous iterations don’t accumulate.

    loss.backward() # Performs backpropagation to calculate the new gradients of the loss with respect to each parameter (weights).
    optimizer.step() # Uses the calculated gradients to update the model's parameters, moving them toward values that minimize the loss.



In [ ]:
# Graph the losses
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epochs')

In [ ]:
# Evaluate the model on test dataset
with torch.no_grad(): # Turn off backpropagation, since learning is not required, only evaluation with test data
    y_eval = model.forward(X_test)
    loss = criterion(y_eval, y_test)
loss


In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = model.forward(data)
        # Will tell us which of the 3 classes has the highest value
        print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

        # Correct or not
        if y_val.argmax().item() == y_test[i]:
            correct += 1

print(f'We got correct {correct} out of {len(y_test)}')

In [ ]:
new_iris = torch.tensor([[4.7, 3.2, 1.3, 0.2]])

In [ ]:
with torch.no_grad():
    y_new = model.forward(new_iris)
    print(y_new, y_new.argmax().item())
    

In [ ]:
# Save our NN Model
torch.save(model.state_dict(), 'basi_nn_iris_model.pt')

In [ ]:
new_model = Model()
new_model.load_state_dict(torch.load('basi_nn_iris_model.pt'))

In [ ]:
new_model.eval() # Put the model in evaluation mode

# Convolutional Neural Networks with PyTorch

**Loading MNIST Dataset for CNNs and classify digits**

image filter nad kerner are the samething.
apply that filter to the image to get feautres. The filter is applied on every matrix, and the number are added up to get a single number. This is called convolution operation.
This filter is dragged over the entire image to get an output

The filter is actually weights. Then it will convvolute and stride across the image.
Sometimes we want to change stride length.

In CNN the neurons are connected to only a small region of the previous layer. This is called local receptive field. --> locally connected.
in ANN, all neurons are connected to all neurons in the previous layer. --> fully connected.

After the local receptive field, which is convolution, there is pooling.

When color images are in usd, the images are 3D and the filters are also 3D. So there is 3d tensors. The coloir is split into multiple layers --> 3 color channel (RGB). Each channel intenstiy determines what color overall the image is.
Each channel is a filter layer.

The very last stage of CNN, there will be a fully connected layer.

Pooling layer:
Takes all the inputs from the convolution layer and reduced them.
This can be done with a regular NN. too, but it touwld take too long and many more parameters.
Reducing that happens with pooling layer.
Pooling is basically downsampling.
Methods to downsample opr pooling: max and average pooling
Drag a filter across, set a window, set a stride length.
max pooling: across the widnows, takes the max values. All the other values disappear.

From pooling layer, a whole new convolution layer can be created.
And then another pooling layer can be created.

Average pooling: The sums are averaged instead of taking the max.

Later they get flattneed out as flatten layer, then a full connected layers to get the output.